# MODEL_TRAININGS

In [1]:
!pip install numpy pandas scikit-learn matplotlib tensorflow tqdm h5py


  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   -- ------------------------------------- 0.8/10.7 MB 6.6 MB/s eta 0:00:02
   ----- -------------

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import tensorflow as tf
from tqdm import tqdm
import h5py

print("All installed successfully!")

All installed successfully!


## STEP 1: Creating PCs for each data

In [4]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import os

# Set paths
data_file = 'HNSCC_DP_data_test.csv'  # file in current folder
output_folder = './'  # Save output in the same folder

# Load the data
df = pd.read_csv(data_file, index_col=0)
print(f"Original shape: {df.shape}")

# Handle missing values
data_clean = df.fillna(0).values  # or use imputation

# Determine PCA components based on number of samples
n_samples = data_clean.shape[0]
n_components = 250 if n_samples < 500 else 500

# Fit PCA
pca = PCA(n_components=n_components)
pca_result = pca.fit_transform(data_clean)
print(f"PCA result shape: {pca_result.shape}")

# Save PCA-transformed matrix (encoded data)
pca_df = pd.DataFrame(pca_result, index=df.index, columns=[f"PC{i+1}" for i in range(n_components)])
pca_df.to_csv(os.path.join(output_folder, f"HNSCC_PCA_{n_components}L.csv"))

# Save PCA components (eigenvectors)
components_df = pd.DataFrame(pca.components_.T, index=df.columns, columns=[f"PC{i+1}" for i in range(n_components)])
components_df.to_csv(os.path.join(output_folder, f"HNSCC_PCA_{n_components}L_COMPONENTS.csv"))

print("✅ PCA encoding and components saved.")


Original shape: (643, 11020)
PCA result shape: (643, 500)
✅ PCA encoding and components saved.


## STEP 2: Training VAE models

In [13]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, BatchNormalization, Activation, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback

# 🔧 Config
input_filename = "HNSCC_PCA_500L.csv"
output_folder = "./vae_outputs/"
latent_dim = 10
intermediate1_dim = 250
intermediate2_dim = 100
epochs = 50
batch_size = 50
cancer_type = "HNSCC"

os.makedirs(output_folder, exist_ok=True)

# 📥 Load Data
df = pd.read_csv(input_filename, index_col=0)
X = df.values
original_dim = X.shape[1]

# 📦 Sampling Layer
class Sampling(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# 🧠 VAE Model Class with KL Loss
class VAE(Model):
    def __init__(self, encoder, decoder, beta=1.0, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = tf.Variable(beta, trainable=False)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)

        # Reconstruction loss
        recon_loss = tf.reduce_mean(tf.reduce_sum(tf.square(inputs - reconstructed), axis=1))
        # KL divergence
        kl = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
        kl_loss = tf.reduce_mean(kl)

        self.add_loss(recon_loss + self.beta * kl_loss)
        return reconstructed

class WarmUpCallback(Callback):
    def __init__(self, beta_var, kappa=0.01):
        super().__init__()
        self.beta_var = beta_var
        self.kappa = kappa

    def on_epoch_end(self, epoch, logs=None):
        if self.beta_var < 1.0:
            self.model.beta.assign(min(1.0, self.beta_var + self.kappa))

# 🏗️ Encoder
x = Input(shape=(original_dim,))
h = Dense(intermediate1_dim)(x)
h = BatchNormalization()(h)
h = Activation("relu")(h)
h = Dense(intermediate2_dim)(h)
h = BatchNormalization()(h)
h = Activation("relu")(h)

z_mean = Dense(latent_dim, name="z_mean")(h)
z_log_var = Dense(latent_dim, name="z_log_var")(h)
z = Sampling()([z_mean, z_log_var])

encoder = Model(inputs=x, outputs=[z_mean, z_log_var, z], name="encoder")

# 🏗️ Decoder
latent_inputs = Input(shape=(latent_dim,))
d = Dense(intermediate2_dim, activation="relu")(latent_inputs)
d = Dense(intermediate1_dim, activation="relu")(d)
decoded = Dense(original_dim)(d)
decoder = Model(inputs=latent_inputs, outputs=decoded, name="decoder")

# 🚀 Compile and Train
vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))
vae.summary()
vae.fit(X, X, epochs=epochs, batch_size=batch_size, shuffle=True, callbacks=[WarmUpCallback(beta_var=vae.beta)])

# 💾 Save Encoded Representation
z_mean, _, _ = encoder.predict(X, batch_size=batch_size)
encoded_df = pd.DataFrame(z_mean, index=df.index)
encoded_df.to_csv(f"{output_folder}/{cancer_type}_VAE_{latent_dim}L_encoding.csv")
print("✅ Latent encoding saved.")

# 📏 Reconstruction & R²
X_recon = decoder.predict(z_mean)
recons_error = mean_squared_error(X, X_recon)
r2_scores = [r2_score(X[i], X_recon[i]) for i in range(X.shape[0])]
print(f"Reconstruction Error: {recons_error:.5f}")
print(f"Mean R² Score: {np.mean(r2_scores):.4f}")


Model: "vae"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape          ┃      Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ encoder (Functional)          │ ((None, 10), (None,   │      153,770 │
│                               │ 10), (None, 10))      │              │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ decoder (Functional)          │ (None, 500)           │      151,850 │
└───────────────────────────────┴───────────────────────┴──────────────┘

 Total params: 305,620 (1.17 MB)

 Trainable params: 304,920 (1.16 MB)

 Non-trainable params: 700 (2.73 KB)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 11099.9990
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10038.0645
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10246.1338
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10165.6934
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 10168.2129
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 10128.8184
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10177.4424
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 9929.4258
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10182.1846
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 9947.3193
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9592.4678
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 9279.9062
Epoch 13/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9148.8965
Epoch 14/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 8986.9932
Epoch 15/50
13/13 ━━━━━━━━━━━━━━━━

In [26]:
output_folder = "./vae_outputs/"
os.makedirs(output_folder, exist_ok=True)

# Save inside vae_outputs/
encoder.save(os.path.join(output_folder, "HNSCC_VAE_10L_encoder.h5"))

## STEP 3: Running IG for VAE models

In [49]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import custom_object_scope

# ---------------------------
# Modern Integrated Gradients Class (TF2+)
# ---------------------------
class IntegratedGradients:
    def __init__(self, model, outchannels=None, verbose=1):
        self.model = model
        if outchannels is None:
            self.outchannels = list(range(self.model.output.shape[1]))
            if verbose:
                print(f"IntegratedGradients: Evaluating all output channels: {self.outchannels}")
        else:
            self.outchannels = outchannels
            if verbose:
                print(f"IntegratedGradients: Evaluating specified output channels: {self.outchannels}")
        if verbose:
            print("IntegratedGradients: Model initialized.")

    def explain(self, sample, outc=0, reference=None, num_steps=50, batch_size=32, verbose=0):
        sample = np.array(sample)
        if reference is None:
            reference = np.zeros_like(sample)

        # Generate interpolation path
        interpolated = self._interpolate(reference, sample, num_steps)  # (num_steps+1, input_dim)

        # Compute gradients at each point
        gradients = self._compute_gradients(interpolated, outc, batch_size, verbose=verbose)  # (num_steps+1, input_dim)

        # Integral approximation: average gradients * (sample - reference)
        avg_grad = np.mean(gradients, axis=0)
        integrated_grads = (sample - reference) * avg_grad
        return integrated_grads

    def _interpolate(self, baseline, sample, num_steps):
        alphas = np.linspace(0, 1, num_steps + 1)
        interpolated = np.array([baseline + alpha * (sample - baseline) for alpha in alphas])
        return interpolated

    def _compute_gradients(self, samples, outc, batch_size=32, verbose=0):
        gradients = []
        n = len(samples)
        for i in tqdm(range(0, n, batch_size), disable=(verbose == 0), desc="Computing gradients"):
            batch = samples[i:i + batch_size]
            inputs = tf.convert_to_tensor(batch, dtype=tf.float32)
            with tf.GradientTape() as tape:
                tape.watch(inputs)
                outputs = self.model(inputs)
                if isinstance(outputs, (list, tuple)):
                    outputs = outputs[0]
                target = outputs[:, outc]
            grads = tape.gradient(target, inputs)
            gradients.append(grads.numpy())
        return np.concatenate(gradients, axis=0)

# ---------------------------
# User-defined parameters
# ---------------------------
cancer = "HNSCC"
latent_dim = 10
n_components = 500
batch_size = 32
steps = 50
input_folder = "./vae_outputs/"

# ---------------------------
# Load PCA-transformed input data
# ---------------------------
print("Loading PCA input data...")
pca_input_file = os.path.join(input_folder, f"{cancer}_PCA_{n_components}L.csv")
X_pca = pd.read_csv(pca_input_file, index_col=0).values
print("PCA input shape:", X_pca.shape)

# ---------------------------
# Load PCA components
# ---------------------------
print("Loading PCA components...")
components_file = os.path.join(input_folder, f"{cancer}_PCA_{n_components}L_COMPONENTS.csv")
components = pd.read_csv(components_file, index_col=0).values
print("PCA components shape:", components.shape)

# ---------------------------
# Load Encoder Model
# ---------------------------
print("Loading encoder model...")
encoder_weights_file = os.path.join(input_folder, f"{cancer}_VAE_{latent_dim}L_encoder.h5")

# If your encoder uses a custom Sampling layer, define it here:
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

class Sampling(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = K.random_normal(shape=K.shape(z_mean))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Load encoder model with custom Sampling layer
with custom_object_scope({'Sampling': Sampling}):
    encoder = load_model(encoder_weights_file, compile=False)

encoder.summary()

# ---------------------------
# Compute Integrated Gradients
# ---------------------------
print("Running Integrated Gradients...")

ig = IntegratedGradients(model=encoder, outchannels=list(range(latent_dim)), verbose=1)

# Allocate attribution array
attributions = np.zeros((latent_dim, X_pca.shape[0], X_pca.shape[1]))  # (latent_dim, n_samples, input_dim)

for node in tqdm(range(latent_dim), desc="Latent nodes"):
    for idx in tqdm(range(X_pca.shape[0]), leave=False, desc="Samples"):
        sample = X_pca[idx]
        baseline = np.zeros_like(sample)
        attr = ig.explain(
            sample=sample,
            outc=node,
            reference=baseline,
            num_steps=steps,
            batch_size=batch_size
        )
        attributions[node, idx, :] = attr

# ---------------------------
# Post-process and Save Results
# ---------------------------
# Average attributions across all samples for each latent node
avg_attr = np.mean(attributions, axis=1)  # (latent_dim, input_dim)

# Project importance scores back to gene space
gene_importance = avg_attr @ components.T  # (latent_dim, n_genes)

# Create DataFrame and save
attr_df = pd.DataFrame(gene_importance, columns=[f"Gene_{i}" for i in range(gene_importance.shape[1])])
attr_df.index = [f"Node_{i}" for i in range(latent_dim)]

out_file = os.path.join(input_folder, f"{cancer}_IG_L{latent_dim}.csv")
attr_df.to_csv(out_file)
print(f"Integrated Gradients saved to: {out_file}")


Loading PCA input data...
PCA input shape: (643, 500)
Loading PCA components...
PCA components shape: (11020, 500)
Loading encoder model...


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)       ┃ Output Shape     ┃   Param # ┃ Connected to     ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ input_layer_5      │ (None, 500)      │         0 │ -                │
│ (InputLayer)       │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ dense_32 (Dense)   │ (None, 250)      │   125,250 │ input_layer_5[0… │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ batch_normalizati… │ (None, 250)      │     1,000 │ dense_32[0][0]   │
│ (BatchNormalizati… │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ activation_10      │ (None, 250)      │         0 │ batch_normaliza… │
│ (Activation)       │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ dense_33 (Dense)   │ (None, 100)      │    25,100 │ activation_10[0… │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ batch_normalizati… │ (None, 100)      │       400 │ dense_33[0][0]   │
│ (BatchNormalizati… │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ activation_11      │ (None, 100)      │         0 │ batch_normaliza… │
│ (Activation)       │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ z_mean (Dense)     │ (None, 10)       │     1,010 │ activation_11[0… │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ z_log_var (Dense)  │ (None, 10)       │     1,010 │ activation_11[0… │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ sampling           │ (None, 10)       │         0 │ z_mean[0][0],    │
│ (Sampling)         │                  │           │ z_log_var[0][0]  │
└────────────────────┴──────────────────┴───────────┴──────────────────┘

 Total params: 153,770 (600.66 KB)

 Trainable params: 153,070 (597.93 KB)

 Non-trainable params: 700 (2.73 KB)

Running Integrated Gradients...
IntegratedGradients: Evaluating specified output channels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
IntegratedGradients: Model initialized.


Latent nodes: 100%|████████████████████████| 10/10 [04:25<00:00, 26.58s/it]


Integrated Gradients saved to: ./vae_outputs/HNSCC_IG_L10.csv


### add functiion

In [48]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

class IntegratedGradients:
    def __init__(self, model, outchannels=None, verbose=1):
        """
        Args:
            model: A tf.keras.Model or keras.Model instance.
            outchannels: List of output node indices to attribute (default: all).
            verbose: Verbosity (default: 1).
        """
        self.model = model
        if outchannels is None:
            # Use all output channels
            self.outchannels = list(range(self.model.output.shape[1]))
            if verbose:
                print(f"IntegratedGradients: Evaluating all output channels: {self.outchannels}")
        else:
            self.outchannels = outchannels
            if verbose:
                print(f"IntegratedGradients: Evaluating specified output channels: {self.outchannels}")
        if verbose:
            print("IntegratedGradients: Model initialized.")

    def explain(self, sample, outc=0, reference=None, num_steps=50, batch_size=32, verbose=0):
        """
        Args:
            sample: np.ndarray of shape (input_dim,) -- a **single** input sample.
            outc: Output channel index to attribute.
            reference: Baseline (default: all zeros, same shape as sample).
            num_steps: Number of steps along interpolation path.
            batch_size: Gradient batch size.
        Returns:
            Integrated gradients attributions, same shape as sample.
        """
        sample = np.array(sample)
        if reference is None:
            reference = np.zeros_like(sample)

        # Generate interpolation path
        interpolated = self._interpolate(reference, sample, num_steps)  # (num_steps+1, input_dim)

        # Compute gradients at each point
        gradients = self._compute_gradients(interpolated, outc, batch_size, verbose=verbose)  # (num_steps+1, input_dim)

        # Integral approximation: average gradients * (sample - reference)
        avg_grad = np.mean(gradients, axis=0)
        integrated_grads = (sample - reference) * avg_grad
        return integrated_grads

    def _interpolate(self, baseline, sample, num_steps):
        # Linear interpolation between baseline and sample
        alphas = np.linspace(0, 1, num_steps + 1)
        interpolated = np.array([baseline + alpha * (sample - baseline) for alpha in alphas])
        return interpolated

    def _compute_gradients(self, samples, outc, batch_size=32, verbose=0):
        # Compute gradients batch-wise
        gradients = []
        n = len(samples)
        for i in tqdm(range(0, n, batch_size), disable=(verbose == 0), desc="Computing gradients"):
            batch = samples[i:i + batch_size]
            inputs = tf.convert_to_tensor(batch, dtype=tf.float32)
            with tf.GradientTape() as tape:
                tape.watch(inputs)
                outputs = self.model(inputs)
                if isinstance(outputs, (list, tuple)):
                    outputs = outputs[0]
                target = outputs[:, outc]
            grads = tape.gradient(target, inputs)
            gradients.append(grads.numpy())
        return np.concatenate(gradients, axis=0)

# Example Usage (assuming encoder is your model, X_pca is your data):

# ig = IntegratedGradients(model=encoder, outchannels=list(range(latent_dim)))
# attr = ig.explain(sample=X_pca[0], outc=0)  # For one sample, one node



In [45]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
import sys
from tqdm import tqdm

class IntegratedGradients:
    def __init__(self, model, outchannels=None, verbose=1):
        self.model = model
        
        # Determine output channels to explain
        if outchannels is None:
            self.outchannels = list(range(model.output.shape[1]))
            if verbose:
                print(f"Evaluating all output channels: {self.outchannels}")
        else:
            self.outchannels = outchannels
            if verbose:
                print(f"Evaluating specified output channels: {self.outchannels}")

        if verbose:
            print("Model initialized for Integrated Gradients")

    def explain(self, sample, outc=0, reference=None, num_steps=50, batch_size=32):
        if outc not in self.outchannels:
            raise ValueError(f"Output channel {outc} not in available channels {self.outchannels}")

        # Prepare baseline (reference)
        if reference is None:
            reference = np.zeros_like(sample)
        
        # Create interpolation path
        interpolated_samples = self._create_interpolation_path(sample, reference, num_steps)
        
        # Compute gradients
        gradients = self._compute_gradients(interpolated_samples, outc, batch_size)
        
        # Integrate gradients
        integrated_grads = (sample - reference) * np.mean(gradients, axis=0)
        
        return integrated_grads

    def _create_interpolation_path(self, sample, reference, num_steps):
        alphas = np.linspace(0, 1, num_steps + 1)
        path = np.array([reference + alpha * (sample - reference) for alpha in alphas])
        return path

    def _compute_gradients(self, samples, outc, batch_size):
        gradients = []
        
        # Process in batches to save memory
        for i in tqdm(range(0, len(samples), batch_size), desc="Computing gradients"):
            batch = samples[i:i + batch_size]
            
            with tf.GradientTape() as tape:
                inputs = tf.convert_to_tensor(batch, dtype=tf.float32)
                tape.watch(inputs)
                outputs = self.model(inputs)
                if isinstance(outputs, list):
                    outputs = outputs[0]  # Take first output if multiple
                outputs = outputs[:, outc]
            
            batch_grads = tape.gradient(outputs, inputs).numpy()
            gradients.append(batch_grads)
        
        return np.concatenate(gradients, axis=0)

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import custom_object_scope
from tqdm import tqdm

# Import custom Integrated Gradients class
from IntegratedGradients import integrated_gradients

# ---------------------------
# User-defined parameters
# ---------------------------
cancer = "HNSCC"
latent_dim = 10
n_components = 500
batch_size = 32
steps = 50
input_folder = "./vae_outputs/"

# ---------------------------
# Load PCA-transformed input data
# ---------------------------
print("Loading PCA input data...")
pca_input_file = os.path.join(input_folder, f"{cancer}_PCA_{n_components}L.csv")
X_pca = pd.read_csv(pca_input_file, index_col=0).values
print("PCA input shape:", X_pca.shape)

# ---------------------------
# Load PCA components
# ---------------------------
print("Loading PCA components...")
components_file = os.path.join(input_folder, f"{cancer}_PCA_{n_components}L_COMPONENTS.csv")
components = pd.read_csv(components_file, index_col=0).values
print("PCA components shape:", components.shape)

# ---------------------------
# Load Encoder Model
# ---------------------------
print("Loading encoder model...")
encoder_weights_file = os.path.join(input_folder, f"{cancer}_VAE_{latent_dim}L_encoder.h5")

# Define custom Sampling layer if needed by encoder
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

class Sampling(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = K.random_normal(shape=K.shape(z_mean))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Load encoder model with custom Sampling layer
with custom_object_scope({'Sampling': Sampling}):
    encoder = load_model(encoder_weights_file, compile=False)

encoder.summary()

# ---------------------------
# Compute Integrated Gradients
# ---------------------------
# Compute Integrated Gradients
print("Running Integrated Gradients...")

# Initialize IntegratedGradients
ig = IntegratedGradients(model=encoder, outchannels=list(range(latent_dim)), verbose=1)

# Compute IG for all samples and all latent nodes
attributions = np.zeros((latent_dim, X_pca.shape[0], X_pca.shape[1]))  # (latent_dim, n_samples, input_dim)
for node in tqdm(range(latent_dim), desc="Latent nodes"):
    for idx in tqdm(range(X_pca.shape[0]), leave=False, desc="Samples"):
        sample = X_pca[idx]  # (input_dim,)
        baseline = np.zeros_like(sample)  # (input_dim,)
        attr = ig.explain(
            sample=sample,
            outc=node,
            reference=baseline,
            num_steps=50,
            batch_size=32
        )
        attributions[node, idx, :] = attr

# Convert to numpy array
attributions = np.array(attributions)

# Shape: (latent_dim, input_dim)
avg_attr = np.mean(attributions, axis=1)

# Project importance scores back to gene space
gene_importance = avg_attr @ components.T

# Create DataFrame and save
attr_df = pd.DataFrame(gene_importance, columns=[f"Gene_{i}" for i in range(gene_importance.shape[1])])
attr_df.index = [f"Node_{i}" for i in range(latent_dim)]

out_file = os.path.join(input_folder, f"{cancer}_IG_L{latent_dim}.csv")
attr_df.to_csv(out_file)
print(f"Integrated Gradients saved to: {out_file}")


Loading PCA input data...
PCA input shape: (643, 500)
Loading PCA components...
PCA components shape: (11020, 500)
Loading encoder model...


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)       ┃ Output Shape     ┃   Param # ┃ Connected to     ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ input_layer_5      │ (None, 500)      │         0 │ -                │
│ (InputLayer)       │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ dense_32 (Dense)   │ (None, 250)      │   125,250 │ input_layer_5[0… │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ batch_normalizati… │ (None, 250)      │     1,000 │ dense_32[0][0]   │
│ (BatchNormalizati… │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ activation_10      │ (None, 250)      │         0 │ batch_normaliza… │
│ (Activation)       │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ dense_33 (Dense)   │ (None, 100)      │    25,100 │ activation_10[0… │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ batch_normalizati… │ (None, 100)      │       400 │ dense_33[0][0]   │
│ (BatchNormalizati… │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ activation_11      │ (None, 100)      │         0 │ batch_normaliza… │
│ (Activation)       │                  │           │                  │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ z_mean (Dense)     │ (None, 10)       │     1,010 │ activation_11[0… │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ z_log_var (Dense)  │ (None, 10)       │     1,010 │ activation_11[0… │
├────────────────────┼──────────────────┼───────────┼──────────────────┤
│ sampling           │ (None, 10)       │         0 │ z_mean[0][0],    │
│ (Sampling)         │                  │           │ z_log_var[0][0]  │
└────────────────────┴──────────────────┴───────────┴──────────────────┘

 Total params: 153,770 (600.66 KB)

 Trainable params: 153,070 (597.93 KB)

 Non-trainable params: 700 (2.73 KB)

Running Integrated Gradients...
Evaluating specified output channels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Model initialized for Integrated Gradients


Samples:   0%|                                     | 0/643 [00:00<?, ?it/s]

Computing gradients:   0%|                           | 0/2 [00:00<?, ?it/s]

Computing gradients: 100%|███████████████████| 2/2 [00:00<00:00,  5.29it/s]

Samples:   0%|                             | 1/643 [00:00<04:12,  2.54it/s]

Computing gradients: 100%|███████████████████| 2/2 [00:00<00:00, 45.25it/s]


Computing gradients: 100%|███████████████████| 2/2 [00:00<00:00, 50.02it/s]


Computing gradients: 100%|███████████████████| 2/2 [00:00<00:00, 51.54it/s]

Samples:   1%|▏                            | 4/643 [00:00<01:11,  8.90it/s]

Computing gradients: 100%|███████████████████| 2/2 [00:00<00:00, 48.78it/s]


Computing gradients: 100%|███████████████████| 2/2 [00:00<00:00, 46.75it/s]


Computing gradients: 100%|███████████████████| 2/2 [00:00<00:00, 50.10it/s]

Samples:   1%|▎                            | 7/643 [00:00<00:48, 13.00it/s]

Computing gradients: 100%|███████████████████| 2/2 [00:00<00:00, 52.51it

In [29]:
import os
print(os.listdir("./vae_outputs"))


['.ipynb_checkpoints', 'HNSCC_PCA_500L.csv', 'HNSCC_PCA_500L_COMPONENTS.csv', 'HNSCC_VAE_10L_encoder.h5', 'HNSCC_VAE_10L_encoding.csv']


In [50]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
